<p align="center"><img src="../additionals/university-workshops-ytu-banner-en.png" alt="University Workshops" style="display: block; margin: 0 auto" height=/></p>

## Welcome to University Workshops!

Welcome to Empa Electronics' first University Workshop event, Yıldız Technical University Workshop!  

This code script will provide you a test environment for the Human Activity Recognition application. 
  

**Steps in the Test Environment:**

1. Requirements

2. Import Trained Model

3. Read & Predict Samples from Data Stream

## 1. Requirements

Importing the required modules:

In [ ]:

import tensorflow as tf
import serial
import numpy as np

## 2. Import Trained Models

Loading the trained model:

In [ ]:
model = tf.keras.models.load_model("../models/model_cnn_workshop_HAR_0_96_v3.h5")

Defining the class mapping dictionary:

In [8]:
# class ID to class name mapping
classes = {0: "STANDING",
           1: "WALKING",
           2: "RUNNING"}

## 3. Read & Predict Samples from Data Stream

Setting the serial connection configuration:

In [9]:
serial_port = '/dev/ttyACM0'
baud_rate = 115200
time_out = 2

Running the data sampling over serial connection & predicting the obtained data sample:

In [ ]:
# collect data, get inference from model
while True:
    # open the serial port, set baudrate, set timeout
    with serial.Serial(serial_port, baud_rate, timeout=time_out) as ser_read:
        # read the line from serial port
        x = ser_read.readline()
        # parse the line into numpy array
        line = np.array(str(x).replace("b'", "").replace("\\n'", "").replace("\\r", "").split(" ")[:-1])
        # ignore the line if the length is not 6*128=768
        if len(line) == 768:
            # convert the array of strings to array of floating point numbers
            line = line.astype(np.float32)
            # reshape the array for model
            line = line.reshape(128,6)
            # add a batch shape to the array
            line = np.expand_dims(line, axis=0)
            # get the model output
            out = model(line)
            # model outputs a tensor, convert it to numpy array
            out = np.array(out)
            # get the highest scoring output as predicted class
            class_index = np.argmax(out)
            # get the class name from dict
            class_name = classes[class_index]
            # print the results
            print(f"\rProbabilities -> Standing: {out[0,0]:.2f} Walking: {out[0,1]:.2f} Running: {out[0,2]:.2f} --- Prediction: {class_name} ",end="")